In [80]:
# Propagation from AZ/EL observations
# Perturbation:
# N/A
# Maneuver: 
# No

# IOD
# Given from propagation_issX

# Obserbation:
# AZ/EL
# TOD / IERS_2010

# Output:
# TBD


In [81]:
%matplotlib inline

from math import radians, pi
import pandas as pd
import numpy as np
import plotly.express as px


import orekit
vm = orekit.initVM()

from orekit.pyhelpers import setup_orekit_curdir, absolutedate_to_datetime
setup_orekit_curdir()


from org.orekit.orbits import KeplerianOrbit, PositionAngle
from org.orekit.propagation.analytical import KeplerianPropagator
from org.orekit.time import AbsoluteDate, TimeScalesFactory, TimeScale
from org.orekit.utils import Constants
from org.orekit.frames import FramesFactory, TopocentricFrame
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.attitudes import LofOffset
from org.orekit.bodies import CelestialBodyFactory, OneAxisEllipsoid
from org.orekit.forces.drag import IsotropicDrag
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.maneuvers import ImpulseManeuver
from org.orekit.frames import FramesFactory, LOFType, EOPHistory
from org.orekit.models.earth.atmosphere import HarrisPriester
from org.orekit.orbits import CircularOrbit, OrbitType, PositionAngle
from org.orekit.propagation import PropagationType, SpacecraftState
from org.orekit.propagation.events import DateDetector, EventEnablingPredicateFilter, PythonEnablingPredicate,\
     PositionAngleDetector, PythonEventDetector, AbstractDetector, EventDetector
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder, DSSTPropagatorBuilder, \
                        ClassicalRungeKuttaIntegratorBuilder, EulerIntegratorBuilder
from org.orekit.propagation.semianalytical.dsst.forces import DSSTAtmosphericDrag, DSSTNewtonianAttraction, DSSTZonal
from org.orekit.time import AbsoluteDate, TimeScalesFactory
from org.orekit.utils import Constants, IERSConventions, PVCoordinatesProvider
from orekit.pyhelpers import absolutedate_to_datetime
from org.hipparchus.linear import RealMatrix
from org.hipparchus.linear import QRDecomposer, AbstractRealMatrix
from org.hipparchus.optim.nonlinear.vector.leastsquares import GaussNewtonOptimizer
from org.orekit.estimation.leastsquares import BatchLSEstimator
from org.orekit.propagation.numerical import NumericalPropagator
from org.orekit.propagation.conversion import AbstractPropagatorBuilder

from org.orekit.estimation.measurements import AngularAzEl
from org.orekit.estimation.measurements import ObservableSatellite
from org.orekit.estimation.measurements import GroundStation
from org.orekit.estimation.sequential import KalmanEstimatorBuilder
from org.orekit.estimation.sequential import KalmanEstimator
from org.orekit.estimation.sequential import KalmanEstimation,AbstractCovarianceMatrixProvider
from org.orekit.estimation.measurements import *
from org.orekit.estimation.iod import IodGooding, IodLaplace

from orekit.pyhelpers import absolutedate_to_datetime, datetime_to_absolutedate, JArray_double2D
from collections import namedtuple
from org.orekit.time import Month
from orekit import JArray

from org.orekit.estimation.sequential import KalmanEstimator, KalmanEstimatorBuilder, CovarianceMatrixProvider, ConstantProcessNoise
from org.orekit.orbits import Orbit
from org.hipparchus.linear import MatrixUtils

In [82]:
utc = TimeScalesFactory.getUTC()

# Given by ADS
a = 6000000.0  
e = 0.2 
i = radians(98.0)     
omega = radians(0.2)   # perigee argument
raan = radians(90.0)  # right ascension of ascending node
lv = radians(0.0)#0.1)    # True anomaly

#mass = 625.0

epochDate = AbsoluteDate(2020, 1, 1, 0, 0, 00.000, utc)
initialDate = epochDate

## Inertial frame where the satellite is defined
inertialFrame =  FramesFactory.getTOD(IERSConventions.IERS_2010, False)
earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                         Constants.WGS84_EARTH_FLATTENING, 
                         inertialFrame)

## Orbit construction as Keplerian
initialOrbit = KeplerianOrbit(a, e, i, omega, raan, lv,
                              PositionAngle.TRUE,
                              inertialFrame, epochDate, Constants.WGS84_EARTH_MU)

utc = TimeScalesFactory.getUTC()

# Orbit propagator parameters
prop_min_step = 60.0 # s
prop_max_step = 300.0 # s
prop_position_error = 100.0 # m

# Estimator parameters
estimator_position_scale = 1.0 # m
estimator_convergence_thres = 1e-3
estimator_max_iterations = 25
estimator_max_evaluations = 35

moon = CelestialBodyFactory.getMoon()
sun = CelestialBodyFactory.getSun()

In [83]:
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder
integratorBuilder = DormandPrince853IntegratorBuilder(prop_min_step, prop_max_step, prop_position_error)
#integratorBuilder = EulerIntegratorBuilder(10.0)

from org.orekit.propagation.conversion import NumericalPropagatorBuilder
from org.orekit.orbits import PositionAngle
propagatorBuilder = NumericalPropagatorBuilder(initialOrbit,
                                               integratorBuilder, PositionAngle.MEAN, estimator_position_scale)
#propagatorBuilder.setMass(1000.0)
#propagatorBuilder.setAttitudeProvider(nadirPointing)

In [84]:
# Earth gravity field with degree 20 and order 20
from org.orekit.forces.gravity.potential import GravityFieldFactory
gravityProvider = GravityFieldFactory.getConstantNormalizedProvider(20,20)
from org.orekit.forces.gravity import HolmesFeatherstoneAttractionModel
gravityAttractionModel = HolmesFeatherstoneAttractionModel(inertialFrame, gravityProvider)
propagatorBuilder.addForceModel(gravityAttractionModel)

# Moon and Sun perturbations
#from org.orekit.forces.gravity import ThirdBodyAttraction
#moon_3dbodyattraction = ThirdBodyAttraction(moon)
#propagatorBuilder.addForceModel(moon_3dbodyattraction)
#sun_3dbodyattraction = ThirdBodyAttraction(sun)
#propagatorBuilder.addForceModel(sun_3dbodyattraction)

# Solar radiation pressure
#from org.orekit.forces.radiation import IsotropicRadiationSingleCoefficient
#isotropicRadiationSingleCoeff = IsotropicRadiationSingleCoefficient(sat_list[sc_name]['cross_section'], sat_list[sc_name]['cr']);
#from org.orekit.forces.radiation import SolarRadiationPressure
#solarRadiationPressure = SolarRadiationPressure(sun, wgs84Ellipsoid.getEquatorialRadius(),
#                                                isotropicRadiationSingleCoeff)
#propagatorBuilder.addForceModel(solarRadiationPressure)

# Atmospheric drag
#from org.orekit.forces.drag.atmosphere.data import MarshallSolarActivityFutureEstimation
#msafe = MarshallSolarActivityFutureEstimation(
#    '(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\p{Digit}\p{Digit}\p{Digit}\p{Digit}F10\.(?:txt|TXT)',
#    MarshallSolarActivityFutureEstimation.StrengthLevel.AVERAGE)
#DM.feed(msafe.getSupportedNames(), msafe) # Feeding the F10.7 bulletins to Orekit's data manager

#from org.orekit.forces.drag.atmosphere import NRLMSISE00
#atmosphere = NRLMSISE00(msafe, sun, wgs84Ellipsoid)
#from org.orekit.forces.drag.atmosphere import DTM2000
#atmosphere = DTM2000(msafe, sun, wgs84Ellipsoid)
#from org.orekit.forces.drag import IsotropicDrag
#isotropicDrag = IsotropicDrag(sat_list[sc_name]['cross_section'], sat_list[sc_name]['cd'])
#from org.orekit.forces.drag import DragForce
#dragForce = DragForce(atmosphere, isotropicDrag)
#propagatorBuilder.addForceModel(dragForce)

# Relativity
#from org.orekit.forces.gravity import Relativity
#relativity = Relativity(orekit_constants.EIGEN5C_EARTH_MU)
#propagatorBuilder.addForceModel(relativity)

In [85]:
def initCov(initialCov, initData):
    for i in range(0,6):
        initialCov.setEntry(i, i, 1.0);
        for j in range(0,i):
            initialCov.setEntry(i, j, initData[j]);
            initialCov.setEntry(j, i, initData[j]);
            #print(initData[j])
    return initialCov

plst = propagatorBuilder.getPropagationParametersDrivers()

#FBuilder = KalmanEstimatorBuilder()
#plst = propagatorBuilder.getOrbitalParametersDrivers()

from org.orekit.utils import ParameterDriver
dragCoefficient = ParameterDriver('Cd', 2.0, 1.0, 1.0, 3.0)
dragCoefficient.setSelected(True)
plst.add(dragCoefficient)

rpCoeffReflection = ParameterDriver('Cr', 1.5, 1.0, 1.0, 2.0)
rpCoeffReflection.setSelected(True)
plst.add(rpCoeffReflection)

estmDMCAcceleration = ParameterDriver('DMC', 0.0, 1.0, -1.0e-3, -1.0e-3)
estmDMCAcceleration.setSelected(True)
plst.add(estmDMCAcceleration)

# Attitude Provider == NULL asumption

# Creation of Noise matrix
Q = MatrixUtils.createRealDiagonalMatrix([1e-0, 1e-0, 1e-0, 1e-0, 1e-0, 1e-0])
# Creation of initial Covariance matrix
from org.hipparchus.linear import Array2DRowRealMatrix
initialCov = Array2DRowRealMatrix(6,6)
# From Orbdetpy, Settings.java
#     public double[] estmCovariance = new double[]{25E6, 25E6, 25E6, 1E2, 1E2, 1E2, 1.00, 0.25, 1E-6, 1E-6, 1E-6};
initData = [25e6, 25e6, 25e6, 1e2, 1e2, 1e2, 1.00, 0.25, 1e-6, 1e-6,1e-6]


initialCov = initCov(initialCov, initData)
#print(Array2DRowRealMatrix.toString(initialCov))

#for i in range(len(initData)):
#    initialCov.addToEntry(i, i, initData[i])
 

    
from org.orekit.estimation.sequential import KalmanEstimatorBuilder, KalmanEstimation

from org.orekit.orbits import CartesianOrbit, Orbit
from org.orekit.utils import TimeStampedPVCoordinates

#print(CartesianOrbit(initialOrbit).getPVCoordinates())
#print(type(CartesianOrbit(initialOrbit).getPVCoordinates()))

#a = TimeStampedPVCoordinates(initialDate,1.0,Orbit.getPVCoordinates(initialOrbit))
#print(type(a))

kepPropagatorBuilder = NumericalPropagatorBuilder(initialOrbit, 
                                                  integratorBuilder, 
                                                  PositionAngle.MEAN,
                                                  1.0)
# Build the Kalman filter
processNoise = ConstantProcessNoise(initialCov, Q)
kalman = KalmanEstimatorBuilder().addPropagationConfiguration(kepPropagatorBuilder, processNoise)
# Uncomment to concider estimated parameters
#kalman.estimatedMeasurementsParameters(plst)
filter = kalman.build()
print(type(filter))
#from org.orekit.estimation.sequential import KalmanObserver 
#observer = KalmanObserver
#filter.setObserver()



<class 'org.orekit.estimation.sequential.KalmanEstimator'>


In [79]:
# Build ObservedMeasurement
#Open and collect data from csv
file = pd.read_csv("export_all_NoPert.csv")
az = file['azimuth']
el = file['elevation']
datetime = file['pv']
idx = datetime[0].find(',')

errDeg = 0.001 #10*0.000277778 #1 arcsec
# TBD
azBaseWeight = 1.0
elBaseWeight = 1.0

# Load values
# Ground station creation
#frame = FramesFactory.getTOD(False)
#frame = FramesFactory.getITRF(IERSConventions.IERS_2010, False)
frame = FramesFactory.getTOD(IERSConventions.IERS_2010, False)

earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                         Constants.WGS84_EARTH_FLATTENING, 
                         frame)
longitude = radians(21.038)
latitude  = radians(67.8790708)
altitude  = 527.0
station = GeodeticPoint(latitude, longitude, altitude)
stationFrame = TopocentricFrame(earth, station, "Kiruna")


ObsSat= ObservableSatellite(0) # From R5
for j in range(2,6):
    year = int(datetime[j][1:5])
    month = int(datetime[j][6:8])
    day = int(datetime[j][9:11])
    hour = int(datetime[j][12:14])
    minute = int(datetime[j][15:17])
    second = int(datetime[j][18:20])
    date = AbsoluteDate(year, Month.getMonth(month), day, hour, minute, 0.0, utc)#TimeScalesFactory.getUT1(IERSConventions.IERS_2010, False))
    orekitAzEl = AngularAzEl(GroundStation(stationFrame),
					#datetime_to_absolutedate(datetime[j][1:idx]),
                    date,
					JArray('double')([radians(az[j]),radians(el[j])]),
                    JArray('double')([radians(errDeg),radians(errDeg)]),
                    JArray('double')([azBaseWeight,elBaseWeight]),
                    ObsSat)
    filter.estimationStep(orekitAzEl)
    #newEstimatedOrbit = newPropagator.getInitialState().getOrbit()
    newEstimatedState = filter.getPhysicalEstimatedState()
    newCovMatrix = filter.getPhysicalEstimatedCovarianceMatrix()
    a = filter.getCurrentDate()
    print(a,newEstimatedState)
    #print(newEstimatedState)
    #print(newCovMatrix)
    #estimator.addMeasurement(orekitAzEl)
#estimationStep(ObservedMeasurement<?> observedMeasurement)
print(orekitAzEl.getDate())

2020-01-01T00:03:00.000 {6,000,003.96645814; 0.0599912585; 1.7039533612; -0.1226854806; 1.5711441227; 0.368987258}
2020-01-01T00:04:00.000 {6,000,003.979117817; 0.0580155905; 1.7047051862; -0.1259756055; 1.5705672302; 0.4612122974}
2020-01-01T00:05:00.000 {6,000,003.976116219; 0.0553460828; 1.7057803677; -0.1221356412; 1.569690919; 0.5500009355}
2020-01-01T00:06:00.000 {6,000,003.9705101885; 0.0481037309; 1.7043271405; -0.1181379537; 1.5692907862; 0.6389772047}
2020-01-01T00:06:00.000


In [78]:
filter.estimationStep(orekitAzEl)
#newEstimatedOrbit = newPropagator.getInitialState().getOrbit()
newEstimatedState = filter.getPhysicalEstimatedState()
newCovMatrix = filter.getPhysicalEstimatedCovarianceMatrix()
a = filter.getCurrentDate()
print(a)
print(newEstimatedState)
print(newCovMatrix)


2020-01-01T00:06:00.000
{6,000,003.983318713; 0.0387681045; 1.6971391056; -0.0994143063; 1.5715048477; 0.6158459198}
Array2DRowRealMatrix{{-37500990.4432385,-5.8587348985,-0.591959673,-62.6604937635,-0.2505385532,65.4044957213},{-5.8587352868,10.1964373656,9.2234559865,1.7172467083,-2.1011857567,6.0496686721},{-0.5919601994,9.2234559871,15.0582424127,2.228786644,2.3683706635,5.3505528104},{-62.6604944835,1.7172467224,2.228786662,31.0258500576,-0.5808555433,-26.2436116509},{-0.2505385709,-2.1011857566,2.3683706636,-0.5808555438,3.1611168099,-0.7409012846},{65.40449606,6.0496686596,5.3505527941,-26.2436116617,-0.7409012851,27.8043245095}}


In [30]:
filter_log = []
rej_count = []
class filt_step_observer(PythonKalmanObserver):
    def evaluationPerformed(self, est):
        kal_epoch = absolutedate_to_datetime(est.currentDate)
        kal_meas_corr = list(est.getCorrectedMeasurement().getEstimatedValue())
        obs_meas = list(est.getPredictedMeasurement().getObservedValue())

        # Residuals
        resids_corr = [kal-obs for kal, obs in zip(kal_meas_corr, obs_meas)]

        # Kalman process Corrected Measurement Status
        status = est.correctedMeasurement.getStatus().toString()

        # Cd and Cr
        coeff = {i.getName(): i.getValue() for i in est.getEstimatedPropagationParameters().getDrivers()}
        est_cd = coeff['drag coefficient'] if any('drag' in s for s in list(coeff.keys())) else c_d
        est_cr = coeff['reflection coefficient'] if any('reflection' in s for s in list(coeff.keys())) else c_r

        # State Covariance
        est_state_cov_mat = est.getPhysicalEstimatedCovarianceMatrix()
        est_state_covar = [est_state_cov_mat.getEntry(x, x) for x in range(est_state_cov_mat.getRowDimension())]

        # Create the Output Dict
        output_dict = {'Epoch': kal_epoch, 'Status': status, 'Cd': est_cd, 'Cr': est_cr, 'State_Covariance': est_state_covar}

        # Measurement Type
        m_type = est.correctedMeasurement.observedMeasurement.getClass()
        s_type = ['Val1', 'Val2', 'Val3', 'Val4', 'Val5', 'Val6']
        if m_type in (Position.class_, PV.class_):
            output_dict['Tracker'] = 'GPS'
            output_dict['Meas_Type'] = 'NavSol'
            sigma = tle_meas_white_noise_sigma
        else:
            trkr = list(est.correctedMeasurement.observedMeasurement.getParametersDrivers())[0]
            output_dict['Tracker'] = trkr.toString().split('-offset')[0].upper()
            if m_type == Range.class_:
                output_dict['Meas_Type'] = 'Range'
                sigma = range_white_noise_sigma
            elif m_type == RangeRate.class_:
                output_dict['Meas_Type'] = 'Doppler'
                sigma = rangerate_white_noise_sigma
            elif m_type == AngularAzEl.class_:
                output_dict['Meas_Type'] = 'AzEl'
                sigma = angles_white_noise_sigma

        # Add Corrected residuals to output
        for m_pairs in list(zip(s_type, resids_corr)):
            output_dict[m_pairs[0] + '_corr'] = m_pairs[1]
        for m_pairs in list(zip(s_type, resids_corr)):
            output_dict[m_pairs[0] + '_ratios'] = m_pairs[1] / sigma

        # Add to the log
        filter_log.append(output_dict)

        # Update the progressbar
        pbar.update()

        # Exit if filter is diverging
        if status == 'REJECTED':
            rej_count.append(status)
            if len(rej_count) > filter_diverge_threshold:
                print(f'\n\nFATAL ERROR: Filter has rejected {filter_diverge_threshold} measurements in a row - exiting\n', flush=True)
                raise ValueError
        else:
            rej_count.clear()

TypeError: object of type 'Array2DRowRealMatrix' has no len()

In [76]:
from org.hipparchus.linear import Array2DRowRealMatrix
initialCov = Array2DRowRealMatrix(6,6)
# From Orbdetpy, Settings.java
#     public double[] estmCovariance = new double[]{25E6, 25E6, 25E6, 1E2, 1E2, 1E2, 1.00, 0.25, 1E-6, 1E-6, 1E-6};
initData = [25e6, 25e6, 25e6, 1e2, 1e2, 1e2]
for i in range(len(initData)):
    InitialCov.addToEntry(i, i, initData[i])

In [77]:
initialOrbit
initialCov
mesurementsFrame

<Array2DRowRealMatrix: Array2DRowRealMatrix{{25000000.0,0.0,0.0,0.0,0.0,0.0},{0.0,25000000.0,0.0,0.0,0.0,0.0},{0.0,0.0,25000000.0,0.0,0.0,0.0},{0.0,0.0,0.0,100.0,0.0,0.0},{0.0,0.0,0.0,0.0,100.0,0.0},{0.0,0.0,0.0,0.0,0.0,100.0}}>

In [73]:
# R14
def performEKF(mesurementsFrame, initialOrbit, initialCOV):
    # Init lists to get the evolution of the filter
    listPropagators = np.array([])
    listOrbits = np.array([])
    listStates = np.array([])
    listCOVmatrixes = np.array([])

    # Init process noise matrix
    Q = MatrixUtils.createRealDiagonalMatrix([1e-0, 1e-0, 1e-0, 1e-0, 1e-0, 1e-0])

    # Init Keplerian Propagator Builder
    integratorBuilder = DormandPrince853IntegratorBuilder(PROP_MIN_STEP, PROP_MAX_STEP, PROP_POS_TOLERANCE)
    kepPropagatorBuilder = NumericalPropagatorBuilder(CartesianOrbit(orbitEstimate), integratorBuilder, PositionAngle.MEAN, estimator_position_scale)

    # Build the Kalman filter
    kalman = KalmanEstimatorBuilder().addPropagationConfiguration(kepPropagatorBuilder,
             ConstantProcessNoise(initialCOV, Q)).estimatedMeasurementsParameters(ParameterDriversList()).build()

    # Process filtering
    for date, meas in mesurements.iterrows():
        kalman.estimationStep(meas['rangeMes'])
        kalman.estimationStep(meas['rangeRateMes'])
        newPropagator = kalman.estimationStep(meas['angularAzElMes'])[0]

    newEstimatedOrbit = newPropagator.getInitialState().getOrbit()
    newEstimatedState = kalman.getPhysicalEstimatedState()
    newCovMatrix = kalman.getPhysicalEstimatedCovarianceMatrix()

    listPropagators = np.append(listPropagators, newPropagator)
    listOrbits = np.append(listOrbits, newEstimatedOrbit)
    listStates = np.append(listStates, newEstimatedState)
    listCovmatrixes = np.append(listCovmatrixes, kalman.getPhysicalEstimatedCovarianceMatrix())

    return listPropagators, listOrbits, listStates, listCovmatrixes

In [ ]:
ovarianceMatricesProvider = UnivariateProcessNoise(cov, LOFType.TNW, angle, lofCartesianOrbitalParametersEvolution, propagationParametersEvolution);
        

In [ ]:
def RealMatrix getInitialCovariance():
    {
	int states = parameters.size() + 6;
	RealMatrix cov = MatrixUtils.createRealIdentityMatrix(states);
	for (int i = 0, k = 0; i < states; i++)
	{
	    if (2*estmCovariance.length == states*(states + 1))
	    {
		// Initialize with the given lower triangular entries
		for (int j = 0; j <= i; j++, k++)
		{
		    cov.setEntry(i, j, estmCovariance[k]); 
		    cov.setEntry(j, i, estmCovariance[k]); 
		}
	    }
	    else
	    {
		// Initialize with the given diagonal entries
		if (i < estmCovariance.length)
		    cov.setEntry(i, i, estmCovariance[i]); 
	    }
	}
	return(cov);
    }

In [5]:
matrixDecomposer = QRDecomposer(1e-11)
optimizer = GaussNewtonOptimizer(matrixDecomposer, False)


estimator = BatchLSEstimator(optimizer, propagatorBuilder)
estimator.setParametersConvergenceThreshold(estimator_convergence_thres)
estimator.setMaxIterations(estimator_max_iterations)
estimator.setMaxEvaluations(estimator_max_evaluations)

In [6]:
# Fetching range data

#Open and collect data from csv
file = pd.read_csv("export_all_NoPert.csv")
az = file['azimuth']
el = file['elevation']
datetime = file['pv']
idx = datetime[0].find(',')

errDeg = 0.001 #10*0.000277778 #1 arcsec
# TBD
azBaseWeight = 1.0
elBaseWeight = 1.0

# Load values
# Ground station creation
#frame = FramesFactory.getTOD(False)
#frame = FramesFactory.getITRF(IERSConventions.IERS_2010, False)
frame = FramesFactory.getTOD(IERSConventions.IERS_2010, False)

earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS, 
                         Constants.WGS84_EARTH_FLATTENING, 
                         frame)
longitude = radians(21.038)
latitude  = radians(67.8790708)
altitude  = 527.0
station = GeodeticPoint(latitude, longitude, altitude)
stationFrame = TopocentricFrame(earth, station, "Kiruna")


ObsSat= ObservableSatellite(0) # From R5
for j in range(0,100):
    year = int(datetime[j][1:5])
    month = int(datetime[j][6:8])
    day = int(datetime[j][9:11])
    hour = int(datetime[j][12:14])
    minute = int(datetime[j][15:17])
    second = int(datetime[j][18:20])
    date = AbsoluteDate(year, Month.getMonth(month), day, hour, minute, 0.0, utc)#TimeScalesFactory.getUT1(IERSConventions.IERS_2010, False))
    orekitAzEl = AngularAzEl(GroundStation(stationFrame),
					#datetime_to_absolutedate(datetime[j][1:idx]),
                    date,
					JArray('double')([radians(az[j]),radians(el[j])]),
                    JArray('double')([radians(errDeg),radians(errDeg)]),
                    JArray('double')([azBaseWeight,elBaseWeight]),
                    ObsSat)
    estimator.addMeasurement(orekitAzEl)



In [7]:
# Orbit estimation
estimatedPropagatorArray = estimator.estimate()

In [8]:
estimatedPropagator = estimatedPropagatorArray[0]
estimatedInitialState = estimatedPropagator.getInitialState()
actualOdDate = estimatedInitialState.getDate()
estimatedOrbit_init = estimatedInitialState.getOrbit()

In [9]:
print(estimatedPropagator)
print(estimatedInitialState)
print(actualOdDate)
print(estimatedOrbit_init)
print(estimatedPropagator.getEventsDetectors())

org.orekit.propagation.numerical.NumericalPropagator@435871cb
SpacecraftState{orbit=Keplerian parameters: {a: 7010886.04754482; e: 0.20064153008167762; i: 97.99328934019404; pa: 0.30265287986772743; raan: 89.96355526364127; v: 0.05653148171348855;}, attitude=org.orekit.attitudes.Attitude@4bef0fe3, mass=1000.0, additional={}}
2020-01-01T00:00:00.000
Keplerian parameters: {a: 7010886.04754482; e: 0.20064153008167762; i: 97.99328934019404; pa: 0.30265287986772743; raan: 89.96355526364127; v: 0.05653148171348855;}
[]


In [11]:
# Run the filter
print(f'\nFilter starting at {meas.get(0).date.toString()}', flush=True)
with tqdm(total=meas.size(), desc='Running the Filter') as pbar:
    try:
        final_state = filt.processMeasurements(meas)
    except:
        write_ephem = False
        filter_out_path = False
        predict_time = False
        show_rejected = True

NameError: name 'meas' is not defined

In [12]:
filter_log = []
rej_count = []
class filt_step_observer(PythonKalmanObserver):
    def evaluationPerformed(self, est):
        kal_epoch = absolutedate_to_datetime(est.currentDate)
        kal_meas_corr = list(est.getCorrectedMeasurement().getEstimatedValue())
        obs_meas = list(est.getPredictedMeasurement().getObservedValue())

        # Residuals
        resids_corr = [kal-obs for kal, obs in zip(kal_meas_corr, obs_meas)]

        # Kalman process Corrected Measurement Status
        status = est.correctedMeasurement.getStatus().toString()

        # Cd and Cr
        coeff = {i.getName(): i.getValue() for i in est.getEstimatedPropagationParameters().getDrivers()}
        est_cd = coeff['drag coefficient'] if any('drag' in s for s in list(coeff.keys())) else c_d
        est_cr = coeff['reflection coefficient'] if any('reflection' in s for s in list(coeff.keys())) else c_r

        # State Covariance
        est_state_cov_mat = est.getPhysicalEstimatedCovarianceMatrix()
        est_state_covar = [est_state_cov_mat.getEntry(x, x) for x in range(est_state_cov_mat.getRowDimension())]

        # Create the Output Dict
        output_dict = {'Epoch': kal_epoch, 'Status': status, 'Cd': est_cd, 'Cr': est_cr, 'State_Covariance': est_state_covar}

        # Measurement Type
        m_type = est.correctedMeasurement.observedMeasurement.getClass()
        s_type = ['Val1', 'Val2', 'Val3', 'Val4', 'Val5', 'Val6']
        if m_type in (Position.class_, PV.class_):
            output_dict['Tracker'] = 'GPS'
            output_dict['Meas_Type'] = 'NavSol'
            sigma = tle_meas_white_noise_sigma
        else:
            trkr = list(est.correctedMeasurement.observedMeasurement.getParametersDrivers())[0]
            output_dict['Tracker'] = trkr.toString().split('-offset')[0].upper()
            if m_type == Range.class_:
                output_dict['Meas_Type'] = 'Range'
                sigma = range_white_noise_sigma
            elif m_type == RangeRate.class_:
                output_dict['Meas_Type'] = 'Doppler'
                sigma = rangerate_white_noise_sigma
            elif m_type == AngularAzEl.class_:
                output_dict['Meas_Type'] = 'AzEl'
                sigma = angles_white_noise_sigma

        # Add Corrected residuals to output
        for m_pairs in list(zip(s_type, resids_corr)):
            output_dict[m_pairs[0] + '_corr'] = m_pairs[1]
        for m_pairs in list(zip(s_type, resids_corr)):
            output_dict[m_pairs[0] + '_ratios'] = m_pairs[1] / sigma

        # Add to the log
        filter_log.append(output_dict)

        # Update the progressbar
        pbar.update()

        # Exit if filter is diverging
        if status == 'REJECTED':
            rej_count.append(status)
            if len(rej_count) > filter_diverge_threshold:
                print(f'\n\nFATAL ERROR: Filter has rejected {filter_diverge_threshold} measurements in a row - exiting\n', flush=True)
                raise ValueError
        else:
            rej_count.clear()